In [1]:
import torch
from torch import nn
from torch.nn import functional as F

import sys
sys.path.append("/home/gleb/MADE/detection-adaptation/src")

import dalib

from contextlib import nullcontext

import pytorch_lightning as pl

In [2]:
pl.seed_everything(0)

0

In [3]:
dam = dalib.models.DetectionAdaptationModule({
    "detector_lr": 3e-6,
    "discriminator_lr": 3e-4,
    "discriminator_pretraining_steps": 500,
    "adaptation_factor": 0.3,
    "normalize_discriminator_loss_by_scores": True,
    "split_discriminator_batch": True,
    "discriminator": {
        "type": "pixelwise",
        "config": {
            "hidden_channels": [128, 128, 128]
        }
    }
})

In [4]:
dam.detector.load_state_dict(torch.load("weights/resnet18-model-002.pth"))

<All keys matched successfully>

In [5]:
dm = dalib.datasets.DetectionDataModule("datasets", config={
    "datasets": [
        {"name": "widerface", "weight": 1.0},
        {"name": "facemask", "weight": 1.0}
    ],
    "val_samples": 512,
    "batch_size": 48
})

for X, y_true in dm.train_dataloader():
    break

y_pred = dam(X)

dam._compute_discriminator_loss(y_pred, y_true)

dam._compute_detector_adaptation_loss(y_pred, y_true)

In [6]:
checkpoint_callback = pl.callbacks.ModelCheckpoint(
    'adapt_experiment/exp',
    monitor='target_val_AP@50',
    mode='max',
    save_last='True'
)

In [7]:
trainer = pl.Trainer(
    checkpoint_callback=checkpoint_callback,
    #checkpoint_callback=None,
    gpus=1,
    automatic_optimization=False,
    val_check_interval=0.1,
    log_every_n_steps=1,
    #max_steps=3300,
    #resume_from_checkpoint="adapt_experiment/exp-v5.ckpt"
)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [ ]:
trainer.fit(dam, datamodule=dm)

Preparing validation mini-dataset: 100%|██████████| 512/512 [00:04<00:00, 113.00it/s]

  | Name          | Type                   | Params
---------------------------------------------------------
0 | detector      | Detector               | 13 M  
1 | discriminator | PixelwiseDiscriminator | 443 K 
/home/gleb/anaconda3/envs/pytorch/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 24 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)
